## Design

1. You need to launch lambda gpus manually. Way too risky for to do that automatically.
2. Then you run the scripts in each cluster.
3. This script will take care of making sure gpus aren't trolling and distribute resources accordingly. Also kills gpus once they're done with their task.
4. tasks it needs to run gets appended to a task queue (which it queries from s3 accordingly using the directory of the file)
2. it processes tasks in batches of M. After each batch is complete, it sends "OK" to another json. It also updates its task queue from s3 (deletes these 20, and takes in more if more were added onto it)
3. If it fails for any task, it sents "NOT OK" and the tasks on its taskqueue to s3 and script stops. Then batch_manager will kill that gpu and redistribute the tasks to an alive gpu.
4. Once a gpu's task queue is empty, it dies.
    - if the last gpu fails early, it'll tell batch manager that there were some tasks left over. then you just need to rerun those tasks when you wake up
    - so the system won't be perfect, but it's risk free, and cost efficient, and it does make it a whole lot less annoying.
    - also later on you can increase risk tolerance (say it only dies early if it fails some percentage of jobs, and failed jobs get added to some dead letter queue. But I'm not adding that in right now. (the dead letter queue exists though)

**I guess there'll be a dedicated company aws account for this... if it becomes something you'd want to set up for multiple people**

**But not mine though**

In [33]:
import pandas as pd
import os
import json
import numpy as np
from collections import defaultdict
import json
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
import requests
import time

In [53]:
global_timeout = 3600*1 #this hard limits your expenditures (3600*1 = 1 hour)

In [124]:
os.environ['AWS_ACCESS_KEY_ID'] = 'key1'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'key2'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
!aws sts get-caller-identity


An error occurred (InvalidClientTokenId) when calling the GetCallerIdentity operation: The security token included in the request is invalid.


In [116]:
#SET THIS MANUALLY EACH TIME
alive_workers = [
    "64.181.251.255",
    "146.235.239.254",
    "64.181.239.13",
    "165.1.78.78",
    "64.181.237.190",
    "146.235.221.170",
    "152.70.121.217"
]

batch_size = 7
worker_2_batch = {}
for i in range(batch_size):
    worker_2_batch[alive_workers[i]] = i

In [125]:
API_KEY = 'lambda_api_key'
BASE_URL = 'https://cloud.lambdalabs.com/api/v1'

In [118]:
import requests

def download_file_to_dict(bucket_name, object_name):
    try:
        response = s3.get_object(Bucket=bucket_name, Key=object_name)
        content = response['Body'].read().decode('utf-8')
        data_dict = json.loads(content)
        return data_dict
    except (NoCredentialsError, PartialCredentialsError):
        print('Credentials not available.')
    except Exception as e:
        print(f'An error occurred: {e}')
        return None


def upload_dict_to_s3(bucket_name, object_name, dictionary):
    json_data = json.dumps(dictionary)
    s3.put_object(Bucket=bucket_name, Key=object_name, Body=json_data)


def move_s3_object(source_bucket_name, source_object_name, dest_bucket_name, dest_object_name):
    try:
        copy_source = {'Bucket': source_bucket_name, 'Key': source_object_name}
        s3.copy_object(CopySource=copy_source, Bucket=dest_bucket_name, Key=dest_object_name)
        s3.delete_object(Bucket=source_bucket_name, Key=source_object_name)
    except (NoCredentialsError, PartialCredentialsError):
        print('Credentials not available.')
    except Exception as e:
        print(f'An error occurred: {e}')


def list_instances():
    url = f'{BASE_URL}/instances'
    headers = {
        'Authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json'
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        instances = response.json()
        return instances
    else:
        print(f'Failed to list instances. Status code: {response.status_code}, Response: {response.text}')
        return None
instances = list_instances()

In [119]:
cluster2id = {}
for ele in instances["data"]:
    cluster2id[worker_2_batch[ele["ip"]]] = ele["id"]

In [120]:
cluster2id

{0: '42b3ee0e2848418eaf79cef318ef3bdd',
 1: '941492dcac5f43daa8ebf6fbebc26f9c',
 2: 'c07ca923c8a84ff1abc32629cff449b6',
 3: '26619e38d69f49f6b92a6eaf4dfc4bc9',
 4: '98827e70798b4752ac869d549c47237d',
 5: 'bffbf93c9494494384b3bd4271952ef9',
 6: '29934d5c96ae43c2a5b8c6723c15b80c'}

In [121]:
import requests
import json

def terminate_instance(cluster_num):
    BASE_URL = 'https://cloud.lambdalabs.com/api/v1/instance-operations/terminate'
    td = {
      "instance_ids": [
        cluster2id[cluster_num]
      ]
    }
    headers = {
        'Content-Type': 'application/json'
    }
    response = requests.post(
        BASE_URL,
        auth=(API_KEY, API_KEY),  # API key is used as the username
        headers=headers,
        json=td  # Payload from the INSTANCE-IDS file
    )
    if response.status_code == 200:
        print(json.dumps(response.json(), indent=2))
    else:
        print(f'Failed to terminate instances. Status code: {response.status_code}, Response: {response.text}')

bucket_name = "rapper-vkg"

In [122]:
live_clusters = list(worker_2_batch.values())
print(live_clusters)
s3 = boto3.client('s3')

[0, 1, 2, 3, 4, 5, 6]


In [123]:
s = time.time()
while(live_clusters):
    print("BATCH MANAGER IS WORKING")
    time.sleep(60)
    lazy_guys = []
    failures = []
    busy_guys = []
    
    for cluster_num in live_clusters:
        tq = download_file_to_dict(f"{bucket_name}-{cluster_num}", f"task_queue_{cluster_num}.json")
        temp_status = tq.get("status", None)
        print(f"status of cluster: {cluster_num}: {temp_status}")
        if tq.get("status", None) == "NOT OK":
            print(f"cluster number {cluster_num} failed")
            failures.append(cluster_num)
        elif tq.get("status", None) == "COMPLETE":
            print(f"cluster number {cluster_num} complete")
            lazy_guys.append(cluster_num)
        else:
            busy_guys.append(cluster_num)
            
    if len(failures) > 0 and (len(lazy_guys)+len(busy_guys)) > 0: #distribute tasks uniformly across all lazy and busy guys.
        print("REDISTRIBUTING TASKS")
        failure_agg_task_queue = []
        nonfailure_task_queue_dict = {}
        for f in failures:
            tf = download_file_to_dict(f"{bucket_name}-{f}", f"task_queue_{f}.json")["task_queue"]
            for t in tf:
                failure_agg_task_queue.append((f, t))
        for l in lazy_guys:
            nonfailure_task_queue_dict[l] = download_file_to_dict(f"{bucket_name}-{l}", f"task_queue_{l}.json")["task_queue"]
        for b in busy_guys:
            nonfailure_task_queue_dict[b] = download_file_to_dict(f"{bucket_name}-{b}", f"task_queue_{b}.json")["task_queue"]
        ftasklen = len(failure_agg_task_queue)
        num_distributed = len(nonfailure_task_queue_dict)
        print(f"Redistributing {ftasklen} tasks to {num_distributed} workers")
        #distributes the tasks and files
        for fatq in failure_agg_task_queue:
            minc = -1
            minval = 9999999999999 #a very fat number
            for m in nonfailure_task_queue_dict:
                if len(nonfailure_task_queue_dict[m]) < minval:
                    minc = m
                    minval = len(nonfailure_task_queue_dict[m])
            assert(minc != -1) #should never happen, but just in case
            nonfailure_task_queue_dict[minc].append(fatq[1])
            move_s3_object(f"{bucket_name}-{fatq[0]}", fatq[1], f"{bucket_name}-{minc}", fatq[1])
        
        #updates the task queue jsons
        for a in nonfailure_task_queue_dict:
            print(f"Updated task_queue_{a}.json")
            upload_dict_to_s3(f"rapper-vkg-{a}", f"task_queue_{a}.json", {"task_queue": nonfailure_task_queue_dict[a]})

        #kill failures
        for f in failures:
            terminate_instance(f)
            
        live_clusters = list(set(live_clusters)-set(failures))
    elif len(failures) > 0: #can't do shit. yer screwed.
        #i didn't fully test this case: if this happens imma have to write more code anyway. global timeout should take care.
        print("THE FORT HAS BEEN BREACHED. THERE IS NO HOPE. THE EMPIRE HAS FALLEN.")
        print("ok but in all seriousness, you have tasks on deadletter queue. you need to redistribute manually, or figure out what's wrong with your system.")
        failure_agg_task_queue_dict = defaultdict(list)
        for f in failures:
            tf = download_file_to_dict(f"{bucket_name}-{f}", f"task_queue_{f}.json")["task_queue"]
            for t in tf:
                failure_agg_task_queue_dict[f].append(t)
        with open("failed_tasks_dead_letter_queue.json", 'w') as file: 
        	json.dump(failure_agg_task_queue_dict, file, indent=4)
        for f in failures:
            terminate_instance(f)

    else: #kill all lazy guys
        if len(lazy_guys) > 0:
            print("KILLING USELESS LAZY GUYS")
            for l in lazy_guys:
                terminate_instance(l)
            live_clusters = list(set(live_clusters)-set(lazy_guys))
        
    e = time.time()
    if e-s > global_timeout:
        print("GLOBAL TIMEOUT REACHED")
        break
    print("BRANCH MANAGER DIDN'T DO ANYTHING")
if live_clusters:
    print("GLOBAL TIMEOUT REACHED, TERMINATING EVERYONE")
    for l in live_clusters:
        terminate_instance(l)
print("DONE!")

BATCH MANAGER IS WORKING
status of cluster: 0: None
status of cluster: 1: None
status of cluster: 2: None
status of cluster: 3: None
status of cluster: 4: None
status of cluster: 5: None
status of cluster: 6: None
BRANCH MANAGER DIDN'T DO ANYTHING
BATCH MANAGER IS WORKING
status of cluster: 0: None
status of cluster: 1: None
status of cluster: 2: None
status of cluster: 3: None
status of cluster: 4: None
status of cluster: 5: None
status of cluster: 6: None
BRANCH MANAGER DIDN'T DO ANYTHING
BATCH MANAGER IS WORKING
status of cluster: 0: None
status of cluster: 1: None
status of cluster: 2: None
status of cluster: 3: None
status of cluster: 4: None
status of cluster: 5: None
status of cluster: 6: None
BRANCH MANAGER DIDN'T DO ANYTHING
BATCH MANAGER IS WORKING
status of cluster: 0: None
status of cluster: 1: None
status of cluster: 2: None
status of cluster: 3: None
status of cluster: 4: None
status of cluster: 5: None
status of cluster: 6: None
BRANCH MANAGER DIDN'T DO ANYTHING
BATCH MA